# Project

### Goal: Improve our original estimate of the log error by using clustering methodologies.

## Acquisition, Prep, and Initial Exploration

Using the notebook and files you created during the exercises make any changes, additions, etc. you want at this point. NOTE: You will NOT be splitting into train and test at this point.

In [1]:
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Wrangling
import pandas as pd
import numpy as np

# Exploring
import scipy.stats as stats

# Impute missing values
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Visualizing
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# default pandas decimal number display format
pd.options.display.float_format = '{:20,.2f}'.format
pd.set_option('display.max_columns', None, 'display.max_rows', None)

import acquire
import wrangle_zillow
import wrangle_zillow_r

In [2]:
# zillow = acquire.get_zillow(acquire.sql)
# Save file to csv so I can quit asking the server for this data
# zillow.to_csv('zillow.csv')

In [3]:
zillow = pd.read_csv('zillow.csv', index_col='id')
# drop extra column that comes in from csv files
# zillow = zillow.drop(columns='Unnamed: 0')

In [4]:
orig_rows = zillow.shape[0]
orig_cols = zillow.shape[1]
print (f'There are {orig_rows} rows and {orig_cols} columns in the starting dataframe.')

There are 77575 rows and 67 columns in the starting dataframe.


In [5]:
# Test wrangle_zillow function

In [21]:
zillow = wrangle_zillow.wrangle_zillow()

In [ ]:
zillow.to_csv('zillow_reduced.csv')

In [19]:
new_rows = zillow.shape[0]
new_cols = zillow.shape[1]
print (f'There are {orig_rows} rows and {orig_cols} columns in the starting dataframe.')
print (f'There are {new_rows} rows and {new_cols} columns in the modified dataframe.')

There are 77575 rows and 67 columns in the starting dataframe.
There are 64841 rows and 34 columns in the modified dataframe.


## Exploration with Clustering

### Cluster the Target Variable

Perform clustering with logerror as the only feature used in the clustering algorithm. Decide on a number of clusters to use, and store the cluster predictions back onto your data frame as cluster_target. Look at the centroids that were produced in this process. What do they tell you?

Use the produced clusters to help you explore through visualization how logerror relates to other variables. (A common way to do this is to use color to indicate the cluster id, and the other variables can be your x-axis and y-axis. (hint: look at your swarmplot function)).

### Cluster Independent Variables

You should also perform some clustering based on a number of independent variables. Create and evaluate several clustering models based on subsets of the independent variables. Here are some ideas:

- Location, that is, latitude and longitude
- Size (finished square feet)
- Location and size

Be sure to use these new clusters in exploring your data, and interpret what these clusters tell you.

### Test the Significance of Clusters
Use statistical testing methods to determine whether the clusters you have created are significant in terms of their relationship to logerror.

## Modeling

### Feature Engineering

Remove variables that are not needed, wanted, useful, or are redundant.

Add any features you think may be useful.

In [ ]:
# Age = yearbuilt - 2017
zillow['age'] = 2017 - zillow.yearbuilt

# Tax Rate = taxamount / 
zillow['tax_rate'] = zillow.taxamount / zillow.taxvaluedollarcnt

# $/sqft
zillow['cost_sf'] = zillow.taxamount / zillow.
zillow['cost_structure'] = zillow.structuretaxvaluedollarcnt / zillow.

# Avg tax rate by censustractandblockid

# $/sqft structure only
# $/sqft lot only
# Avg $/sqft by censustractandblockid


Split your data into training and test sets.

In [ ]:
#train, test = wrangle_zillow.split_my_data(df, train_pct)

# what about scaling?

Create subsets of data if you would like to create multiple models and then merge (such as, a different model for each cluster or for each county).

## Model Selection

Train at least 3 different models (a model is different if there are changes in one or more of the following: features, hyper-parameters, algorithm). Create object, fit, predict & evaluate. Use mean absolute error or mean squared error to evaluate. Also, try regression algorithms you have not used before.

Evaluate your best model on your test data set to get an idea of your model's out of sample error.